# MLCQ Dataset

In [2]:
import numpy as np 
import pandas as pd
import os

In [3]:
def sevrity_to_numeric(severity):
    if severity == 'none':
        return 0
    elif severity == 'minor':
        return 1
    elif severity == 'major':
        return 2
    elif severity == 'critical':
        return 3
    else:
        print(f'sev: {severity}')
        raise Exception 

In [4]:
def combine_severities(df, sample):
    rows = df[df.sample_id == sample]
    return [i for i in rows.severity.array]

In [10]:
mlcq = pd.read_csv('mlcq.csv').sort_values(by="sample_id")
samples = mlcq.drop_duplicates("sample_id").sample_id
# severity 제외 공통된 부분만 픽
mlcq = mlcq.drop(["reviewer_id","review_timestamp"],axis=1)
mlcq.severity = mlcq.severity.map(lambda s: sevrity_to_numeric(s))
severities = samples.map(lambda sample: combine_severities(mlcq, sample))

mlcq["severities"] = pd.Series(severities)
mlcq = mlcq.drop(["severity"], axis=1)
mlcq = mlcq.drop_duplicates(["sample_id"])
mlcq["avg"] = mlcq.severities.map(lambda ss: np.mean(ss))
mlcq

,id,sample_id,smell,type,code_name,repository,commit_hash,path,start_line,end_line,link,is_from_industry_relevant_project,severities,avg
1364,1902,3698323,data class,class,com.amazon.ask.dispatcher.request.handler.impl...,git@github.com:alexa/alexa-skills-kit-sdk-for-...,bf1e9ccc50d1f3f8408f887f70197ee288fd4bd9,/ask-sdk-core/src/com/amazon/ask/dispatcher/re...,26,59,https://github.com/alexa/alexa-skills-kit-sdk-...,1.0,"[0, 0]",0.000000
1357,1895,3698602,feature envy,function,com.amazon.ask.request.mapper.impl.BaseRequest...,git@github.com:alexa/alexa-skills-kit-sdk-for-...,bf1e9ccc50d1f3f8408f887f70197ee288fd4bd9,/ask-sdk-runtime/src/com/amazon/ask/request/ma...,79,81,https://github.com/alexa/alexa-skills-kit-sdk-...,1.0,"[0, 0]",0.000000
1359,1897,3698665,feature envy,function,com.amazon.ask.builder.impl.AbstractSkillBuild...,git@github.com:alexa/alexa-skills-kit-sdk-for-...,bf1e9ccc50d1f3f8408f887f70197ee288fd4bd9,/ask-sdk-runtime/src/com/amazon/ask/builder/im...,91,94,https://github.com/alexa/alexa-skills-kit-sdk-...,1.0,"[0, 0]",0.000000
1355,1893,3698860,long method,function,com.alibaba.android.arouter.facade.model.Route...,git@github.com:alibaba/ARouter.git,93b328569bbdbf75e4aa87f0ecf48c69600591b2,/arouter-annotation/src/main/java/com/alibaba/...,200,213,https://github.com/alibaba/ARouter/blob/93b328...,1.0,"[0, 0]",0.000000
1362,1900,3699227,long method,function,com.alibaba.android.arouter.facade.enums.#getC...,git@github.com:alibaba/ARouter.git,93b328569bbdbf75e4aa87f0ecf48c69600591b2,/arouter-annotation/src/main/java/com/alibaba/...,32,34,https://github.com/alibaba/ARouter/blob/93b328...,1.0,"[0, 0]",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7110,7689,9540354,blob,class,org.apache.hadoop.yarn.server.nodemanager.cont...,git@github.com:apache/hadoop.git,128dd91e10080bdcbcd7d555fa3c4105e55a6b51,/hadoop-yarn-project/hadoop-yarn/hadoop-yarn-s...,24,39,https://github.com/apache/hadoop/blob/128dd91e...,1.0,"[0, 0]",0.000000
10206,10808,9542671,data class,class,org.apache.hadoop.yarn.webapp.hamlet2.HamletSp...,git@github.com:apache/hadoop.git,128dd91e10080bdcbcd7d555fa3c4105e55a6b51,/hadoop-yarn-project/hadoop-yarn/hadoop-yarn-c...,1772,1820,https://github.com/apache/hadoop/blob/128dd91e...,1.0,"[0, 0, 0, 2, 0, 0]",0.333333
4781,5352,9542718,feature envy,function,org.apache.hadoop.mapred.NotRunningJob#killTas...,git@github.com:apache/hadoop.git,128dd91e10080bdcbcd7d555fa3c4105e55a6b51,/hadoop-mapreduce-project/hadoop-mapreduce-cli...,213,219,https://github.com/apache/hadoop/blob/128dd91e...,1.0,"[0, 0]",0.000000
9135,9733,9548458,blob,class,org.apache.hadoop.metrics2.sink.ganglia.Gangli...,git@github.com:apache/hadoop.git,128dd91e10080bdcbcd7d555fa3c4105e55a6b51,/hadoop-common-project/hadoop-common/src/main/...,46,253,https://github.com/apache/hadoop/blob/128dd91e...,1.0,"[0, 0]",0.000000


In [28]:
none_count = len(mlcq[mlcq.avg == 0])
print(f'whole no smells : {none_count / len(mlcq) * 100:2.3}% ({none_count} / {len(mlcq)})')
data = mlcq[mlcq.smell == "long method"]
lm_cnt = len(data[data.avg > 0])
print(f'long methods smell : {lm_cnt / len(data) * 100:2.3}% ({lm_cnt} / {len(data)})')

whole no smells : 74.2% (3538 / 4770)
long methods smell : 19.5% (245 / 1258)
